In [17]:
import networkx as nx
import numpy as np
import igl

mesh = igl.read_triangle_mesh("cube.obj")


  o cube


In [45]:
# returns the midpoints of the edges of a triangle t
def calc_edge_midpoints(t): return (t[0] + t[1])/2, (t[1] + t[2])/2, (t[2] + t[0])/2

# returns the center of a triangle t as vertex to mesh m
def add_center_vertex(m,t): return [m.add_vertex(numpy.mean(t,0))]
# returns duplicate vertices of a list if they exist on a mesh
def get_duplicate_vertices (m,l): return [find_duplicate_vertex(m,v) for v in l]
# if there exists a vertex v in mesh m, it returns that handle, otherwise it returns the handle of the vertex itself 
def find_duplicate_vertex(m,v): return next((m.vertex_handle(i) for i,point in enumerate(m.points()) if all(point == m.point(v)) and v.idx() != i),m.vertex_handle(v.idx()))
# returns a face handle from three elements of a list of vertices
def face_from_list(m,l,a,b,c): return [m.add_face(l[a],l[b],l[c])]
#triangulates the mesh where every triangle is turned into 6 smaller triangles
def clean_mesh (m):
    m.delete_isolated_vertices() 
    m.garbage_collection()
# returns the node that contains the current face
def get_node_of_face (f,nodes,graph_map): return next(node for node in nodes if f in graph_map[node])
# returns the adjacent nodes to node n in tree T         
def get_adjacent_nodes(T,n): return nx.descendants_at_distance(T,n,1)
# returns the faces that map to the nodes 
def get_faces_on_nodes(graph_map,nodes): return [graph_map[n] for n in nodes]
# returns the id of faces in mesh m that are adjacent to face f
def get_adjacent_faces(m,f): return [face.idx() for face in m.ff(m.face_handle(f))]
# sets face f on mesh m to visited
def set_visited(m,f): return m.set_face_property("visited", m.face_handle(f), True)
# checks if face f on mesh m is visited
def is_visited (m,f): return m.face_property("visited", m.face_handle(f))    
# adds nodes to graph G for every face in mesh m
def add_nodes_from_mesh(G,m): [G.add_node(f.idx()) for f in m.faces()]
# adds edges to graph G for every face in mesh m adjacent to another face
def add_edges_from_mesh(G,m): [[G.add_edge(face,ajd_face) for ajd_face in get_adjacent_faces(m,face)] for face,fh in enumerate(m.faces())]  
# get the ids of a list of openMesh handles
def get_ids(l): return [e.idx() for e in l]
# create spanning tree from mesh m using networkx
def spanning_tree_from_mesh(m): return nx.minimum_spanning_tree(graph_from_mesh(m))
# get the triangle representation of a list of faces
def faces_to_triangles (m,faces): return [face_to_triangle(m,face) for face in faces]
# get the triangle representation of a facehandle
def face_to_triangle (m,face):return [(m.point(vertex)) for vertex in m.fv(m.face_handle(face))]
# returns the face from adj_faces that lies on the node returns False if there is no such face
def get_adjacent_face_on_node(m,adj_faces,faces_on_node): return next((face for face in adj_faces if is_visited(m,face) != True and face in faces_on_node), False)


# returns a graph based on mesh data
def graph_from_mesh(m):
    G = nx.Graph()
    add_nodes_from_mesh(G,m)
    add_edges_from_mesh(G,m)
    return G


# returns the triangulated mesh and a dictionary to map the original mesh's triangles to their triangulated counterparts
def triangulate_mesh(mesh):
    v, f = mesh
    
    # TODO properly initiate these two fields
    v_out = v
    f_out = f
    
    tri_dict = {}
    for original_face in f:
        # populate the triangle with points of vertices of the original face
        triangle = [v[vertex_id] for vertex_id in original_face]
        
        # add the 7 vertices to trianglulated mesh that make up the new triangles
        # (3 outer points, center, and 3 midpoints of the edges)
        vertices = [
            # triangle vertices
            triangle[0],
            triangle[1],
            triangle[2],
            
            # center vertex
            np.mean(triangle, axis=0),
            
            # center vertices on the edges
            np.mean([triangle[0], triangle[1]], axis=0),
            np.mean([triangle[1], triangle[2]], axis=0),
            np.mean([triangle[2], triangle[0]], axis=0)
        ]

        # TODO do we really need this?
        # check for duplicate vertices, if found, replace with original
        # vertices = get_duplicate_vertices(triangulated_mesh, vertices)
        
        # TODO check if libigl has merging meshes functionality
        
        # create the six triangulated faces--vertices have to be in cw order
        triangulated_faces = [
            [4, 3, 0],
            [1, 3, 4],
            [5, 3, 1],
            [2, 3, 5],
            [6, 3, 2],
            [0, 3, 6],
        ]
        
        # offset the new vertex ids by the amount of prexisting vertices
        f_out = np.append(f_out, np.array(triangulated_faces) + len(v_out), axis=0)
        v_out = np.append(v_out, vertices, axis=0)

        # remove vertices that used to be duplicates
        # clean_mesh (triangulated_mesh)
        # add the faces to the dictionary
        #tri_dict[original_face.idx()] = get_ids(triangulated_faces)
    return (v_out, f_out), tri_dict

# traversal function. l = list of triangles, m = mesh, T = spanning Tree
def traverse(m, T, graph_map, node, face,l=[]):
    if is_visited(m,face): return l # stop when we return to a visited face
    set_visited(m,face)
    l.append(face)
    adj_nodes = get_adjacent_nodes(T,node)
    adj_faces = get_adjacent_faces(m,face)
    faces_on_other_nodes = flatten_list(get_faces_on_nodes(graph_map,adj_nodes))
    faces_on_this_node = graph_map[node]
    # find the next face: an adjacent face on another node OR the next on the current node
    face = get_adjacent_face_on_node(m,adj_faces,faces_on_other_nodes) or get_adjacent_face_on_node(m,adj_faces,faces_on_this_node)
    return traverse(m,T, graph_map, get_node_of_face(face,adj_nodes|{node}, graph_map), face,l)

def get_vertex_graph(v, f):
    graph = nx.Graph()
    graph.add_nodes_from(range(len(v)))
    graph.add_edges_from(igl.edges(f))
    return graph

def get_face_graph(f):
    face_adjacency_list, _ = igl.triangle_triangle_adjacency(f)
    return nx.from_dict_of_lists(dict((i, adjacency_list) for i, adjacency_list in enumerate(face_adjacency_list)))

# the main algorithm: converts a mesh to a foldable list of triangles
def hamiltonian_refinement(original_mesh):
    triangulated_mesh, graph_map = triangulate_mesh(original_mesh)
    v, f = triangulated_mesh
    face_graph = get_face_graph(f)
    spanning_tree = nx.minimum_spanning_tree(face_graph)
    return traverse(triangulated_mesh, spanning_tree, graph_map,0,0)

print(hamiltonian_refinement(mesh))


AttributeError: 'tuple' object has no attribute 'face_property'